In [64]:
import itertools
import json
from pathlib import Path

import pandas as pd

In [65]:
def sorted_tuple(x):
    return tuple(sorted(x))

In [66]:
from bellem.dvc.experiment import load_experiments

filepaths = list(Path("../../tmp/drop-mini-manual").glob("*.json"))
experiments = [exp for fp in filepaths for exp in load_experiments(fp)]
print(f"{len(experiments)} experiments")
next(iter(experiments), None)

21 experiments


{'commit': 'workspace',
 'id': 'workspace',
 'name': None,
 'params': {'dataset': {'path': 'bdsaglam/drop-musique-mini',
   'name': 'default',
   'split': 'train'},
  'qa': {'model': 'llama-3-8b',
   'temperature': 0.0,
   'system_prompt': 'direct/format-few.txt',
   'user_prompt_template': 'cq.txt',
   'few_shot_examples': 'manual/direct.json',
   'n_shot': 0,
   'n_sc': 1},
  'run': 1},
 'metrics': {'exact_match': 0.37,
  'f1': 0.5993370884585718,
  'fuzzy_match': 0.41,
  '0hops': {'exact_match': 0.37,
   'f1': 0.5993370884585718,
   'fuzzy_match': 0.41},
  'gen_token_count': {'all': {'count': 100.0,
    'mean': 6.41,
    'std': 4.431510219810265,
    'min': 3.0,
    '25%': 4.0,
    '50%': 5.0,
    '75%': 7.0,
    'max': 28.0},
   'success': {'count': 65.0,
    'mean': 5.3076923076923075,
    'std': 2.8718628368789054,
    'min': 3.0,
    '25%': 4.0,
    '50%': 5.0,
    '75%': 5.0,
    'max': 22.0},
   'fail': {'count': 35.0,
    'mean': 8.457142857142857,
    'std': 5.91778405040492

In [67]:
df = pd.json_normalize(experiments)
df.drop(columns=[col for col in df.columns if "fuzzy" in col], inplace=True)
print(f"{len(df)} experiments before preprocessing")
df.head()

21 experiments before preprocessing


,commit,id,name,params.dataset.path,params.dataset.name,params.dataset.split,params.qa.model,params.qa.temperature,params.qa.system_prompt,params.qa.user_prompt_template,...,metrics.gen_token_count.success.75%,metrics.gen_token_count.success.max,metrics.gen_token_count.fail.count,metrics.gen_token_count.fail.mean,metrics.gen_token_count.fail.std,metrics.gen_token_count.fail.min,metrics.gen_token_count.fail.25%,metrics.gen_token_count.fail.50%,metrics.gen_token_count.fail.75%,metrics.gen_token_count.fail.max
0,workspace,workspace,None,bdsaglam/drop-musique-mini,default,train,llama-3-8b,0.0,direct/format-few.txt,cq.txt,...,5.00,22.0,35.0,8.457143,5.917784,3.0,5.0,6.0,9.00,28.0
1,0dc1ac9f99cc3c539cdd54ae3981e8c4f52e6bd3,e7197b9530afa0b74197ac5e6e711e3ccd1d4b0e,hated-fore,bdsaglam/drop-musique-mini,default,train,deepseek-r1-llama-8b,0.5,empty.txt,icq-format.txt,...,441.00,1839.0,59.0,579.779661,1048.349393,169.0,260.5,342.0,439.00,6973.0
2,0dc1ac9f99cc3c539cdd54ae3981e8c4f52e6bd3,39ef92d375302bce71bfecb1c83a632e0dba789f,whity-wave,bdsaglam/drop-musique-mini,default,train,deepseek-r1-llama-8b,0.5,empty.txt,icq-format.txt,...,403.50,764.0,76.0,387.815789,200.796329,149.0,250.0,336.5,470.75,1166.0
3,0dc1ac9f99cc3c539cdd54ae3981e8c4f52e6bd3,de60a8a11f645e0b77cf84ffdabb82aacb3b0123,spicy-bael,bdsaglam/drop-musique-mini,default,train,deepseek-r1-llama-8b,0.5,empty.txt,icq-format.txt,...,403.75,1524.0,60.0,423.283333,332.267816,168.0,255.5,334.5,460.25,2113.0
4,0dc1ac9f99cc3c539cdd54ae3981e8c4f52e6bd3,402223512ca8b9922f56a87af12c575694e2727f,sport-kale,bdsaglam/drop-musique-mini,default,train,deepseek-r1-llama-8b,0.5,empty.txt,icq-format.txt,...,401.00,4019.0,67.0,381.910448,202.044904,159.0,257.0,317.0,403.50,966.0


In [68]:
param_cols = [col for col in df.columns if col.startswith("params.")]
metric_cols = [col for col in df.columns if col.startswith("metrics.")]

In [69]:
df.dropna(subset=metric_cols[:2], inplace=True, how="any")
df.drop_duplicates(subset=param_cols, keep='last', inplace=True)

print(f"{len(df)} experiments after preprocessing")
df.head()

20 experiments after preprocessing


,commit,id,name,params.dataset.path,params.dataset.name,params.dataset.split,params.qa.model,params.qa.temperature,params.qa.system_prompt,params.qa.user_prompt_template,...,metrics.gen_token_count.success.75%,metrics.gen_token_count.success.max,metrics.gen_token_count.fail.count,metrics.gen_token_count.fail.mean,metrics.gen_token_count.fail.std,metrics.gen_token_count.fail.min,metrics.gen_token_count.fail.25%,metrics.gen_token_count.fail.50%,metrics.gen_token_count.fail.75%,metrics.gen_token_count.fail.max
1,0dc1ac9f99cc3c539cdd54ae3981e8c4f52e6bd3,e7197b9530afa0b74197ac5e6e711e3ccd1d4b0e,hated-fore,bdsaglam/drop-musique-mini,default,train,deepseek-r1-llama-8b,0.5,empty.txt,icq-format.txt,...,441.00,1839.0,59.0,579.779661,1048.349393,169.0,260.5,342.0,439.00,6973.0
2,0dc1ac9f99cc3c539cdd54ae3981e8c4f52e6bd3,39ef92d375302bce71bfecb1c83a632e0dba789f,whity-wave,bdsaglam/drop-musique-mini,default,train,deepseek-r1-llama-8b,0.5,empty.txt,icq-format.txt,...,403.50,764.0,76.0,387.815789,200.796329,149.0,250.0,336.5,470.75,1166.0
3,0dc1ac9f99cc3c539cdd54ae3981e8c4f52e6bd3,de60a8a11f645e0b77cf84ffdabb82aacb3b0123,spicy-bael,bdsaglam/drop-musique-mini,default,train,deepseek-r1-llama-8b,0.5,empty.txt,icq-format.txt,...,403.75,1524.0,60.0,423.283333,332.267816,168.0,255.5,334.5,460.25,2113.0
4,0dc1ac9f99cc3c539cdd54ae3981e8c4f52e6bd3,402223512ca8b9922f56a87af12c575694e2727f,sport-kale,bdsaglam/drop-musique-mini,default,train,deepseek-r1-llama-8b,0.5,empty.txt,icq-format.txt,...,401.00,4019.0,67.0,381.910448,202.044904,159.0,257.0,317.0,403.50,966.0
5,0dc1ac9f99cc3c539cdd54ae3981e8c4f52e6bd3,f59054d456e353af61838756bbd55dabeacba25b,loose-grub,bdsaglam/drop-musique-mini,default,train,deepseek-r1-llama-8b,0.5,empty.txt,icq-format.txt,...,462.00,1223.0,71.0,367.338028,213.886815,137.0,236.0,328.0,391.00,1354.0


In [70]:
def parse_qa_technique(row):
    model = row['params.qa.model']
    if 'r1' in model:
        base_technique = 'REASONING'
    else:
        base_technique = row["params.qa.system_prompt"].split("/")[0].replace(".txt", "").upper()
    sc = int(row["params.qa.n_sc"])
    sc_suffix = f"-SC({sc})" if sc > 1 else ""
    return f"{base_technique}{sc_suffix}"

df["params.qa.technique"] = df.apply(parse_qa_technique, axis=1)

In [71]:
def remove_ext(filename):
    return filename.split(".", 1)[0]

def parse_instruction(row):
    sp = remove_ext(row["params.qa.system_prompt"]).lower()
    up = remove_ext(row["params.qa.user_prompt_template"]).lower().replace(".txt", "")
    fs = remove_ext(row["params.qa.few_shot_examples"]).lower()
    return f"{sp}:{up}:{fs}"

df["params.qa.instruction"] = df.apply(parse_instruction, axis=1)

In [72]:
# make temperature 1 decimal
df["params.qa.temperature"] = df["params.qa.temperature"].round(1)

In [73]:
for col in param_cols:
    values = list(df[col].unique())
    print(f"- {col}: {values}")
    print()

- params.dataset.path: ['bdsaglam/drop-musique-mini']

- params.dataset.name: ['default']

- params.dataset.split: ['train']

- params.qa.model: ['deepseek-r1-llama-8b', 'llama-3-8b', 'qwen-2.5-32b']

- params.qa.temperature: [0.5, 0.0]

- params.qa.system_prompt: ['empty.txt', 'direct/format-few.txt', 'ccot/format-thought.txt']

- params.qa.user_prompt_template: ['icq-format.txt', 'cq.txt']

- params.qa.few_shot_examples: ['manual/reasoning.json', 'manual/direct.json', 'manual/ccot.json']

- params.qa.n_shot: [0, 1]

- params.qa.n_sc: [1]

- params.run: [1, 3, 2]



In [74]:
df.to_json('exps.jsonl', orient='records', lines=True)

## Setup remaining experiments

In [75]:
def produce_experiment_configs(common_params, varying_params):
    # Generate all possible combinations of parameters
    varying_params = {**common_params, **varying_params}
    keys = varying_params.keys()
    values = varying_params.values()
    for instance in itertools.product(*values):
        yield dict(zip(keys, instance))

In [76]:
def produce_all_experiment_configs(common_params: dict, varying_params_list: list[dict]):
    for params in varying_params_list:
        for exp_config in produce_experiment_configs(common_params, params):
            yield exp_config

In [77]:
common_params = {
    "params.dataset.path": ["bdsaglam/drop-musique-mini"],
    "params.dataset.split": ["train"],
    "params.qa.temperature": [
        # 0.0,
        0.5,
    ],
    "params.qa.user_prompt_template": ["cq.txt"],
    "params.qa.n_sc": [
        1,
    ],
    "params.qa.n_shot": [
        0,
        1,
    ],
    "params.run": [
        1,
        2,
        3,
    ],
}

In [78]:
varying_params_list = [
    {
        "params.qa.model": [
            "llama-3-8b",
            # "llama-3.3-70b",
        ],
        "params.qa.system_prompt": ["direct/format-few.txt"],
        "params.qa.few_shot_examples": ["manual/direct.json"],
    },
    # {
    #     "params.qa.model": [
    #         "llama-3-8b",
    #         # "llama-3.3-70b",
    #     ],
    #     "params.qa.system_prompt": ["cot/format-thought.txt"],
    #     "params.qa.few_shot_examples": ["manual/cot.json"],
    # },
    {
        "params.qa.model": [
            "llama-3-8b",
            # "llama-3.3-70b",
        ],
        "params.qa.system_prompt": ["ccot/format-thought.txt"],
        "params.qa.few_shot_examples": ["manual/ccot.json"],
    },
    # {
    #     "params.qa.model": [
    #         "llama-3-8b",
    #         # "llama-3.3-70b",
    #     ],
    #     "params.qa.system_prompt": ["cok/adapted.txt"],
    #     "params.qa.few_shot_examples": ["manual/cok.json"],
    # },
    # {
    #     "params.qa.model": [
    #         "llama-3-8b",
    #         # "llama-3.3-70b",
    #     ],
    #     "params.qa.system_prompt": ["cte/format-triples-ere-few.txt"],
    #     "params.qa.few_shot_examples": ["manual/cte-triples.json"],
    # },
    {
        "params.qa.model": [
            "deepseek-r1-llama-8b",
            # "deepseek-r1-llama-70b",
        ],
        "params.qa.system_prompt": ["empty.txt"],
        "params.qa.user_prompt_template": ["icq-format.txt"],
        "params.qa.few_shot_examples": ["manual/reasoning.json"],
    },
]

In [79]:
exp_configs = list(produce_all_experiment_configs(common_params, varying_params_list))
target_params = list(exp_configs[0].keys())
print(f"{len(exp_configs)} experiment configurations")
print(target_params)

18 experiment configurations
['params.dataset.path', 'params.dataset.split', 'params.qa.temperature', 'params.qa.user_prompt_template', 'params.qa.n_sc', 'params.qa.n_shot', 'params.run', 'params.qa.model', 'params.qa.system_prompt', 'params.qa.few_shot_examples']


In [80]:
if len(df):
    existing_configs = df[target_params].to_dict(orient="records")
    existing_configs[0]
else:
    existing_configs = []

print("Existing exps:", len(existing_configs))

Existing exps: 20


In [81]:
# find the missing configurations
missing_configs = [
    dict(kv)
    for kv in list(
        {tuple(sorted(config.items())) for config in exp_configs}
        - {tuple(sorted(config.items())) for config in existing_configs}
    )
]
print(f"{len(missing_configs)} missing configurations")

0 missing configurations


In [82]:
def make_command(exp_config):
    lines = ["dvc exp run --queue"]
    for target_param in target_params:
        arg_name = target_param.split(".", 1)[-1]
        arg_value = exp_config[target_param]
        lines.append(f"-S {arg_name}='{arg_value}'")

    command = " \\\n    ".join(lines)
    return command

In [83]:
with open("run.sh", "w") as f:
    f.write("#!/bin/sh\n\n")
    for exp_config in missing_configs:
        f.write(make_command(exp_config))
        f.write("\n\n")

## Inspect

In [84]:
df

,commit,id,name,params.dataset.path,params.dataset.name,params.dataset.split,params.qa.model,params.qa.temperature,params.qa.system_prompt,params.qa.user_prompt_template,...,metrics.gen_token_count.fail.count,metrics.gen_token_count.fail.mean,metrics.gen_token_count.fail.std,metrics.gen_token_count.fail.min,metrics.gen_token_count.fail.25%,metrics.gen_token_count.fail.50%,metrics.gen_token_count.fail.75%,metrics.gen_token_count.fail.max,params.qa.technique,params.qa.instruction
1,0dc1ac9f99cc3c539cdd54ae3981e8c4f52e6bd3,e7197b9530afa0b74197ac5e6e711e3ccd1d4b0e,hated-fore,bdsaglam/drop-musique-mini,default,train,deepseek-r1-llama-8b,0.5,empty.txt,icq-format.txt,...,59.0,579.779661,1048.349393,169.0,260.50,342.0,439.00,6973.0,REASONING,empty:icq-format:manual/reasoning
2,0dc1ac9f99cc3c539cdd54ae3981e8c4f52e6bd3,39ef92d375302bce71bfecb1c83a632e0dba789f,whity-wave,bdsaglam/drop-musique-mini,default,train,deepseek-r1-llama-8b,0.5,empty.txt,icq-format.txt,...,76.0,387.815789,200.796329,149.0,250.00,336.5,470.75,1166.0,REASONING,empty:icq-format:manual/reasoning
3,0dc1ac9f99cc3c539cdd54ae3981e8c4f52e6bd3,de60a8a11f645e0b77cf84ffdabb82aacb3b0123,spicy-bael,bdsaglam/drop-musique-mini,default,train,deepseek-r1-llama-8b,0.5,empty.txt,icq-format.txt,...,60.0,423.283333,332.267816,168.0,255.50,334.5,460.25,2113.0,REASONING,empty:icq-format:manual/reasoning
4,0dc1ac9f99cc3c539cdd54ae3981e8c4f52e6bd3,402223512ca8b9922f56a87af12c575694e2727f,sport-kale,bdsaglam/drop-musique-mini,default,train,deepseek-r1-llama-8b,0.5,empty.txt,icq-format.txt,...,67.0,381.910448,202.044904,159.0,257.00,317.0,403.50,966.0,REASONING,empty:icq-format:manual/reasoning
5,0dc1ac9f99cc3c539cdd54ae3981e8c4f52e6bd3,f59054d456e353af61838756bbd55dabeacba25b,loose-grub,bdsaglam/drop-musique-mini,default,train,deepseek-r1-llama-8b,0.5,empty.txt,icq-format.txt,...,71.0,367.338028,213.886815,137.0,236.00,328.0,391.00,1354.0,REASONING,empty:icq-format:manual/reasoning
6,0dc1ac9f99cc3c539cdd54ae3981e8c4f52e6bd3,bbff8c0ff1a6ecb898de54478c6887cfe0ddd1be,prone-eric,bdsaglam/drop-musique-mini,default,train,deepseek-r1-llama-8b,0.5,empty.txt,icq-format.txt,...,78.0,340.294872,177.622985,150.0,219.50,298.5,398.50,1127.0,REASONING,empty:icq-format:manual/reasoning
7,workspace,workspace,None,bdsaglam/drop-musique-mini,default,train,llama-3-8b,0.0,direct/format-few.txt,cq.txt,...,35.0,8.457143,5.917784,3.0,5.00,6.0,9.00,28.0,DIRECT,direct/format-few:cq:manual/direct
8,1a7cb4805d827e66a1cf6615af92ff9d2cebd0d3,aeaeef5f62e5a25a1d1a855760ff89be5823fa97,withy-hows,bdsaglam/drop-musique-mini,default,train,llama-3-8b,0.5,ccot/format-thought.txt,cq.txt,...,44.0,33.090909,18.433814,13.0,20.75,28.0,39.50,106.0,CCOT,ccot/format-thought:cq:manual/ccot
9,workspace,workspace,None,bdsaglam/drop-musique-mini,default,train,qwen-2.5-32b,0.0,direct/format-few.txt,cq.txt,...,35.0,8.457143,5.917784,3.0,5.00,6.0,9.00,28.0,DIRECT,direct/format-few:cq:manual/direct
10,b018a25b61d3cc0ae6e1e34779cfc29b7f86eca1,fad0d9ec37347759f0ccfe9d8dfb2a78426b16de,worst-gude,bdsaglam/drop-musique-mini,default,train,llama-3-8b,0.5,direct/format-few.txt,cq.txt,...,40.0,6.950000,4.355662,3.0,4.00,6.0,8.00,25.0,DIRECT,direct/format-few:cq:manual/direct
